# PSTAT 176/276 Project #

Univa Song  
Ziwei Zhang  
Frank Li  
Lingxiao Zhou  
Kyle Guan

## Initialization

In [1]:
# !pip install --upgrade pandas
# !pip install tqdm
# !pip install altair

In [2]:
from tqdm import tqdm
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import scipy
import scipy.sparse as sparse
from scipy.stats import norm
import matplotlib.pyplot as plt

## Problem 1

In [3]:
def StockVol(histoPrice):
    """
    Compute the stock volatility under GBM using 1-year historical prices
    
    Inputs:
        histoPrice: an array of daily historical prices for one year
        
    Returns:
        histoVol: annualized historical volatility
    """
    
    logret = np.diff(np.log(histoPrice))
    sigma = np.sqrt(np.var(logret))
    histoVol = sigma*np.sqrt(252)  # annualize volatility
    
    return histoVol

## Problem 2

In [4]:
def StockPath(n,sigma=0.2,S0=100,T=1,nump=252,r=0.01,delta = 0):
    """
    Generate n stock paths
    
    Inputs:
        n: number of paths generated
        sigma: volatility of the stock
        S0: current stock price
        T: terminal time in yearly unit
        nump: number of time periods
        r: interest rate
        delta: continuous dividend yield of the stock
        
    Returns:
        S: an array of stock paths
    """
    
    X = np.zeros((n,1+nump))
    X[:,0] = S0
    for i in range(len(X)):
        Z = np.random.normal(0, 1, nump)
        X[i,1:]=np.exp(sigma*np.sqrt(T/nump)*Z+(r-delta-sigma**2/2)*(T/nump))
    
    S = []   
    for i in range (n):
        S.append(np.cumprod(X[i,:]))
    
    return np.array(S)

## Problem 3

In [5]:
def EurOptPrice(paths,K,r=0.01,T=1):
    """
    generate the European put option price through Monte Carlo method
    
    Inputs:
        paths: an array of stock paths
        K: strike price
        r: interest rate
        T: terminal time
        
    Returns:
        Payoff: discounted payoffs
        price: estimated price of the European put option
        variance: variance of discounted payoffs
        
    """
    
    Payoff = np.maximum(K-paths[:,-1],0)*np.exp(-r*T)
    price = np.mean(Payoff)
    variance = np.var(Payoff)
    
    return (Payoff,price,variance)
    

## Problem 4

We first simulate *n* stock paths

$$
\begin{bmatrix} 
S_1^{(1)} & S_2^{(1)} & \dots\  S_T^{(1)}\\
\vdots & \vdots &\ddots  \\
S_1^{(n)} & S_2^{(n)} & \dots \ S_T^{(n)} 
\end{bmatrix}
$$

Then we start with $S_T$ and go backwards to compute the holding value $H_i$, early exercised payoff $P_i$ and current option value $V_i$. In order to compute the holding value $H_i = \mathbb{E}[V_{i+1}(S_{i+1})|S_i]$, we use one step Monte Carlo simulation at $S_{i}$ and $V_i = \max(H_i, P_i)$. One challenge is that we need to figure out the option value $V_i$ for different stock prices. So we fit a Random Forest Regression model between $V_i$ and $S_i$ at each step, and use the model to find the option values in the next step. We choose to use Random Forest to build the model because Random Forest,a powerful algorithm in Machine Learning, can be used to solve regression problems, and works well with continuous variables. Also, compared to decision trees, Random Forest reduces the variance and hence improves the accuracy. Finally, we determine the optimal exercising time which is the first time that $P_i$ exceeds $H_i$ and compute the estimated price of the option.

In [6]:
def AmeOptPrice(paths,K,r=0.01,T=1,nump = 252,delta = 0,sigma=0.2):
    """
    generate the American put option price without control variable
    
    Inputs:
        paths: an array of stock paths
        K: strike price
        r: interest rate
        T: terminal time
        nump: number of periods
        
    Returns:
        Payoff: discounted payoffs
        price: estimated price of the American put option
        variance: variance of discounted payoffs
        
    """
    deltaT = T/nump
    P = np.maximum(K-paths,0)  # payoffs if early exercise
    H = np.zeros(paths.shape)  # holding value
    V = np.zeros(paths.shape)  # value of the option
    
    H[:,-1] = P[:,-1]
    V[:,-1] = P[:,-1]
    
    # compute the expected payoff at termial time given S_(T-1) using one step monte carlo
    tmp = paths[:,-2]
    for i in range(len(paths)):
        tmp_Price = StockPath(100,sigma,tmp[i],deltaT,1,r,delta)
        tmp_payoff = np.maximum(K-tmp_Price[:,-1],0)*np.exp(-r*deltaT)
        H[i,-2] = np.mean(tmp_payoff)
    V[:,-2] = np.maximum(P[:,-2], H[:,-2])  # value of the option at t = T-1
    
    rf = RandomForestRegressor(n_estimators=30, n_jobs=-1)  #Define Random Forest Regressor 
    
    for i in range(2,len(V[0])):
        X = paths[:,-i].reshape(-1,1)
        Y = V[:,-i].reshape(-1,1)
        
        reg = rf.fit(X, Y.ravel())  # Polynomial regression (degree = 5)
        
        tmp = paths[:,-i-1]
        for j in range(len(paths)):
            tmp_Price = StockPath(100,sigma,tmp[j],deltaT,1,r,delta)
            tmp_V = rf.predict(tmp_Price.reshape(-1,1))*np.exp(-r*deltaT)
            H[j,-i-1] = np.mean(tmp_V)
        V[:,-i-1] = np.maximum(P[:,-i-1], H[:,-i-1])
 
    # Determine the optimal stopping time and payoffs
    Payoff = [0]*len(P)
    for i in range(len(P)):
        idx = np.where(P[i,:]> H[i,:])[0]
        if(len(idx) == 0):
            Payoff[i] = V[i,-1]*np.exp(-r*T)
        else:
            Payoff[i] = V[i,idx[0]]*np.exp(-r*idx[0]*deltaT)

    price = np.mean(Payoff)
    variance = np.var(Payoff)
    
    return(Payoff, price, variance)    

## Problem 5

The `ConVariate` function can be used to estimate the price of American put option with:
- x = simulated European put payoffs
- y = simulated American put payoffs
- mu_x = true price of the European put option

We also write a function `BSPut` to compute the true European put option price.

In [7]:
def ContVariate(y,x,mu_x):
    """
    Implement the control variates method
    
    Inputs:
        y: an array of samples with unknown mean
        x: an array of samples with known or estimated mean
        mu_x: mean of random variable X
        
    Returns:
        y_hat: estimated mean of y
        y_hatVar: variance of the estimator
        
    """
    
    x_bar = np.mean(x)
    y_bar = np.mean(y)
    y_var = np.var(y)
    corr = np.corrcoef(x,y)[0,1]
    
    beta = beta = np.cov(x,y)[0,1]/np.var(x)
    y_hat = y_bar+beta*(mu_x-x_bar)
    y_hatVar = (np.var(y)/len(y))*(1-corr**2)
    
    return(y_hat[0],y_hatVar)

def BSput(S0, K, T, r, sigma,q):
    """
    Compute true price of the European put using BS formula
    
    Inputs:
        S0: current stock price
        K: strike price
        T: termial time
        r: interest rate
        sigma: volatility
        q: continuous dividend
        
    Returns:
        price: price of the option
    """
    
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = (np.log(S0 / K) + (r - q - 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    
    price = (K * np.exp(-r * T) * norm.cdf(-d2, 0.0, 1.0) - S0 * norm.cdf(-d1, 0.0, 1.0))
    
    return price

## Problem 6: McDonald's Stock Analysis

In this section, we apply AmeOptPrice function to McDonald's Corporation (MCD) to compute the price of MCD210618P00200000 with a strike price 200. We will also calculate the estimated price with control variates and argue that we should use control variates to determine the price of American put options. Then we will use the control variates method to reduce the variance of our estimated American prices.

### Volatility and continuous dividend yield


We use historical prices and dividends between June 5, 2019, and June 3, 2020, to compute the stock volatility and continuous dividend yield. There are 252 trading days in total between June 5, 2019, and June 3, 2020. During this period, MCD paid dividends on Aug 30, 2019, Nov 29, 2019, Feb 28, 2020, and May 29, 2020. The dividends are 1.16, 1.25, 1.25, and 1.25, respectively. The stock prices are 217.97, 194.48, 194.17, and 186.32, respectively.

The calibrated volatility and computed continuous dividend yield are shown below.

In [8]:
import pandas as pd
MCD = pd.read_excel('data/MCD.xlsx')

#extract the LIBOR rate on June 3, 2020 and historical closing prices from the excel 
LIBOR = np.array(pd.DataFrame(MCD, columns=['LIBOR']))
closingPrice = np.array(pd.DataFrame(MCD, columns=['Close']))
histoPrice = np.zeros((1,252))
for i in range(0,252):
    histoPrice[0,i] = closingPrice[i][0]
    
#calculate the interest rate and continuous dividend yield
r = LIBOR[-1]/100
delta = np.log((1+(1.16/217.97))*(1+(1.25/194.48))*(1+(1.25/194.17))*(1+(1.25/186.32)))
sigma = StockVol(histoPrice)

print('The calibrated volatility is:',np.round(sigma,4))
print('The continuous dividend yield is:', np.round(delta,4))

The calibrated volatility is: 0.3972
The continuous dividend yield is: 0.0248


### Estimated price without control variates

We use weekly data (number of period = 52) and 100 paths to compute the estimated price of MCD210618P00200000. We decided to use 100 as stock paths and 52 for the number of periods to reduce computation/run time.

In [9]:
# Simulate Path
paths = StockPath(100,sigma,histoPrice[0][-1],1,52,r[0],delta)

# American Put Option
AmePut = AmeOptPrice(paths,200,r[0],1,52,delta,sigma)
print("American Put Option price:", AmePut[1])
print('Variance of payoffs:',AmePut[2])

American Put Option price: 38.66013686220499
Variance of payoffs: 1177.989163910921


### Estimated price with control variates

We estimate the price of the European put option with the same strike price using the same path and compute the true price by Black-Scholes formula. Then we apply the control variates method to get a more accurate estimated price of the American put option.

In [10]:
# compute the true price of the American put by using the Black-Scholes model 
truePrice = BSput(histoPrice[0][-1], 200, 1, r, sigma, delta)
print("True European Put Price:",truePrice[0])

# compute the price of the European put on MCD
EurPut = EurOptPrice(paths, 200, r, 1)
print("Estimated European Put Option price:", EurPut[1])

# apply the ContVariate function
y_hat,y_hatVar = ContVariate(AmePut[0], EurPut[0], truePrice)
print("Estimated American Put price with control variates:",y_hat)

True European Put Price: 33.60733581303151
Estimated European Put Option price: 38.95612409370057
Estimated American Put price with control variates: 34.788556357468806


The last price, bid price, and ask price of the option we choose are 34.70, 38.75, and 41.45 respectively (All information is from Yahoo Finance). From the results above, we can see that the price of the American put option calculated by control variates is closer to the market price.

### Variance Comparison

If we do not use control variates, the variance of our estimator $\hat Y$, mean of all the payoffs, is  $$Var(\hat Y) = \frac{\sigma^2_Y}{n}$$ where $\sigma^2_Y$ is the variance of payoffs and n is the number of paths simulated. According to results above, we can compute that the variance of our estimator is about 8.4. This is a relatively large number.

Theoratically, the variance of the estimator $\hat Y^*$ computed by the control variates is $$Var(\hat Y^*) = \frac{\sigma^2_Y}{n}[1-corr^2(X,Y)]$$ where X is the payoffs of European put option and Y is the payoffs of American put option. Since $0 \leq corr^2(X,Y) \leq 1$, the new estimator computed under control variates will have a smaller variance.

We compute the estimated prices multiple times to verify that the variance of estimator computed by control variates is smaller than that of estimator computed without control variates. We use monthly data in this part to shorten the running time of the code.

In [11]:
M = 50  # number of runs

AmerP=[0]*M
AmerP_CV=[0]*M
AmerP_var=[0]*M

for i in tqdm(range(M),desc = str(M)+' runs'):
    paths = StockPath(100,sigma,histoPrice[0][-1],1,12,r[0],delta)
    AP_temp=AmeOptPrice(paths,200,r[0],1,12,delta,sigma)
    EP_temp=EurOptPrice(paths, 200, r, 1)
    CV_temp=ContVariate(AP_temp[0], EP_temp[0], truePrice)
    
    AmerP[i] = AP_temp[1]
    AmerP_CV[i] = CV_temp[0]
    AmerP_var[i] = AP_temp[2]

50 runs: 100%|█████████████████████████████████████████████████████████████████████████| 50/50 [11:13<00:00, 13.46s/it]


In [12]:
import pandas as pd
import altair as alt
import numpy as np

source1 = pd.DataFrame({
    'Without Control Variate': AmerP,
    'With Control Variate': AmerP_CV,
})

Chart1 = alt.Chart(source1).transform_fold(
    ['Without Control Variate', 'With Control Variate'],
    as_=['Method', 'Estimated price']
).mark_area(
    opacity=0.3,
    interpolate='step'
).encode(
    alt.X('Estimated price:Q', bin=alt.Bin(maxbins=10)),
    alt.Y('count()', stack=None),
    alt.Color('Method:N')
).properties(
    title='Histogram of estimated prices (50 runs)'
)

source2 = pd.concat([pd.DataFrame({'price': AmerP, 'method':'Without'}),pd.DataFrame({'price': AmerP_CV, 'method':'With'})])

bars = alt.Chart(source2).mark_bar().properties(
    width=alt.Step(80)  # controls width of bar
).encode(
    x='method:N',
    y=alt.Y('mean(price):Q', title='Mean price', scale=alt.Scale(domain=(25,40))),
    #color = 'method:N'
)

error_bars = alt.Chart().mark_errorbar(extent='ci').encode(
    x='method:N',
    y='price:Q'
)

Chart2 = alt.layer(bars, error_bars, data=source2).properties(
    title='Mean of estimated prices (50 runs)'
)

Chart2 | Chart1

alt.HConcatChart(...)

In [13]:
print("The variance of estimator without control vairates is:", np.var(AmerP))
print("The variance of estimator with control vairates is:", np.var(AmerP_CV))

The variance of estimator without control vairates is: 13.438458133032377
The variance of estimator with control vairates is: 2.4495985789767043


We can see that the estimator without control variates has a  greater mean and larger confidence interval (black bar in the first plot). After calculating the variance of each estimator, we observe that the variance of estimator computed under control variates is much smaller than that of the one without. Thus, we believe that applying control variates on top of the AmeOptPrice function is the optimal way to estimate an American option.